In [22]:
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import cv2

def abrir_imagem(caminho):
    img = Image.open(caminho)
    print(f"Formato: {img.format}, Tamanho: {img.size}, Modo: {img.mode}")
    return img, np.array(img)

def plotar_imagem(img_original, img_filtrada, titulo_original, titulo_filtrada):
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

    # Exibindo a imagem original
    ax[0].imshow(img_original, cmap='gray' if img_original.mode == 'L' else None)
    ax[0].set_title(titulo_original)
    ax[0].axis('off')

    # Exibindo a imagem filtrada
    ax[1].imshow(img_filtrada, cmap='gray' if img_filtrada.mode == 'L' else None)
    ax[1].set_title(titulo_filtrada)
    ax[1].axis('off')

    plt.show()

def mensagem_binario(msg):
    if isinstance(msg, str):
        return ''.join([format(ord(i), "08b") for i in msg])
    elif isinstance(msg, (bytes, np.ndarray)):
        return [format(i, "08b") for i in msg]
    elif isinstance(msg, (int, np.uint8)):
        return format(msg, "08b")
    else:
        raise TypeError("Tipo de dado não suportado. Suportado: str, bytes, np.ndarray e int.")

def ocultar_mensagem(img, msg):
    n_bytes = img.size[0] * img.size[1] * 3 // 8
    if len(msg) > n_bytes:
        raise ValueError("A mensagem é muito grande para ser ocultada na imagem.")

    msg += "####"
    dados_binario = mensagem_binario(msg)
    dados_binario = iter(dados_binario)

    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pixel = img.getpixel((i, j))
            pixel_binario = mensagem_binario(pixel)

            try:
                bit = next(dados_binario)
                pixel_binario = pixel_binario[:-1] + bit
                img.putpixel((i, j), int(pixel_binario, 2))
            except StopIteration:
                return

def extrair_mensagem(img):
    dados_binario = []

    for i in range(img.size[0]):
        for j in range(img.size[1]):
            pixel_binario = mensagem_binario(img.getpixel((i, j)))
            dados_binario.append(pixel_binario[-1])  # Pega o bit menos significativo

    num_bytes = len(dados_binario) // 8
    bits_caractere = np.split(np.array(dados_binario[:num_bytes*8]), num_bytes)

    mensagem = ""
    for byte in bits_caractere:
        caractere = chr(int("".join(byte), 2))
        mensagem += caractere

        if mensagem[-5:] == "####":
            return mensagem[:-5]

# Aplicar a transformação logarítmica
def transformacao_logaritmica(img, c):
    img_log = c * np.log1p(img)
    img_log = np.array(np.clip(img_log, 0, 255), dtype=np.uint8)
    return img_log

# Aplicar a transformação de potência (gama)
def transformacao_potencia(img, c, gamma):
    img_pow = c * (img ** gamma)
    img_pow = np.array(np.clip(img_pow, 0, 255), dtype=np.uint8)
    return img_pow

# Implementar a equalização do histograma manualmente
def equalizacao_histograma_manual(img):
    hist, _ = np.histogram(img.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalizado = (255 * cdf / cdf[-1]).astype(np.uint8)
    img_eq = cdf_normalizado[img]
    return img_eq

# Implementar a equalização do histograma com OpenCv
def equalizacao_histograma_opencv(img):
    img_eq = cv2.equalizeHist(img)
    return img_eq

# Implementar a equalização do histograma com Pillow
def equalizacao_histograma_pillow(img):
    img_pillow = Image.fromarray(img)
    img_eq = ImageOps.equalize(img_pillow)
    return np.array(img_eq)

# Representação de cada plano de bits
def representacao_planos_bits(img):
    # Converter a imagem para escala de cinza se não estiver
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Criar 8 planos de bits
    planos_bits = [(img >> i) & 1 for i in range(8)]

    fig, axes = plt.subplots(2, 4, figsize=(12, 6))
    for i, ax in enumerate(axes.flat):
        ax.imshow(planos_bits[i], cmap='gray')
        ax.set_title(f"Plano de bits {i}")
        ax.axis('off')
    plt.show()

def main():
    imagens = ['Fig0308(a)(fractured_spine).tif', 'enhance-me.gif']
    mensagem = "Sua mensagem secreta aqui"

    for caminho in imagens:
        img_original, img_cv = abrir_imagem(caminho)

        if img_original.mode == 'L':  # Somente em tons de cinza
            # Aplicando a transformação logarítmica
            img_log = transformacao_logaritmica(img_cv, c=1)
            plotar_imagem(img_original, Image.fromarray(img_log), "Imagem Original", "Transformação Logarítmica")

            # Aplicando a transformação de potência
            img_pow = transformacao_potencia(img_cv, c=1, gamma=2.0)
            plotar_imagem(img_original, Image.fromarray(img_pow), "Imagem Original", "Transformação de Potência")

            # Equalização de histograma (manual)
            img_eq_manual = equalizacao_histograma_manual(img_cv)
            plotar_imagem(img_original, Image.fromarray(img_eq_manual), "Imagem Original", "Equalização de Histograma Manual")

            # Equalização de histograma (OpenCV)
            img_eq_opencv = equalizacao_histograma_opencv(img_cv)
            plotar_imagem(img_original, Image.fromarray(img_eq_opencv), "Imagem Original", "Equalização de Histograma OpenCV")

            # Equalização de histograma (Pillow)
            img_eq_pillow = equalizacao_histograma_pillow(img_cv)
            plotar_imagem(img_original, Image.fromarray(img_eq_pillow), "Imagem Original", "Equalização de Histograma Pillow")

            # Representação de cada plano de bits
            representacao_planos_bits(img_cv)

        # Esteganografia em tons de cinza e colorido
        img_filtrada = img_cv.copy()
        ocultar_mensagem(Image.fromarray(img_filtrada), mensagem)

        img_filtrada = Image.fromarray(img_filtrada)
        plotar_imagem(img_original, img_filtrada, "Imagem Original", "Imagem com Mensagem Oculta")

        # Extraindo mensagem
        mensagem_extraida = extrair_mensagem(img_filtrada)
        print("Mensagem extraída:", mensagem_extraida)

if __name__ == "__main__":
    main()



Output hidden; open in https://colab.research.google.com to view.